In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import set_option

#import the required libraries for plotting
from matplotlib import pyplot
from pandas.tools.plotting import scatter_matrix

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

#import the required ML libraries
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.preprocessing import LabelEncoder # For categorical variables
from sklearn.preprocessing import OneHotEncoder


#import the performance evaluation modules
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

# Any results you write to the current directory are saved as output.

The following section of code is used to read the files. 

In [ ]:
trainFile = pd.read_csv("../input/train.csv")
print(type(trainFile))
gender_submissionFile = pd.read_csv("../input/gender_submission.csv")
testFile = pd.read_csv("../input/test.csv")
trainFile.head()

Print a list of counts for categorical variables.

In [ ]:
print("List of all the columns: ")
print(trainFile.columns)
print("\n")
print(trainFile['Survived'].value_counts())
print(trainFile['Sex'].value_counts())
print(trainFile['Pclass'].value_counts())
#print(trainFile['Cabin'].value_counts())
print(trainFile['Embarked'].value_counts())




Find and print the correlation between variables

In [ ]:
set_option('display.width', 100)
set_option('precision', 3)
correlations = trainFile.corr(method = 'pearson')
print(correlations['Survived'].sort_values(ascending=False))
sknewness = trainFile.skew()
print(sknewness)

**Data Visualization**

Plot the Histogram for all the attributes of the dataframe trainFile. 

In [ ]:
trainFile.hist(bins = 50,figsize = (20,15))
pyplot.show()

Density Plots: Helps to visualize the data more clearly than the histogram.

In [ ]:
trainFile.plot(kind='density', subplots=True, layout=(3,3), sharex=False, figsize = (15,20))
pyplot.show()

Box and Whisker Plots

In [ ]:
trainFile.plot(kind='box', subplots=True, layout=(3,3), sharex=False, sharey=False,figsize=(15,20))
pyplot.show()

Plotting a scatter plot for Age in x and Fares in y. This plot is to see what age groups of people prefer paying high fares and what prefer paying low fares.   
There is a high density for low fares in the age group between 20 to 40, which means people of this age group prefer paying low fares for the voyage.    
Also, a good correlation between fares and price is seen. The red dots resemble class 3 (lower fare) is present at the bottom of the graph. The blue dots (which resemble class 2 and 3(more bluish)) are present at the top part of the graph. 

In [ ]:
trainFile.plot(kind="scatter", x="Age", y="Fare", alpha=0.4,c="Pclass",cmap=pyplot.get_cmap("jet"),colorbar=True,figsize = (15,10))
pyplot.xlabel("Age")
pyplot.show()

MultiVariate Plots: Correlation Matrix Plot, Scatter Diagram Plot

Correlation Matrix Plot:

In [ ]:
fig = pyplot.figure(figsize = (10,10))
ax = fig.add_subplot(111)
ax.set_xticklabels(trainFile.columns)
ax.set_yticklabels(trainFile.columns)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
pyplot.show()

Scatter Plot:

In [ ]:
scatter_matrix(trainFile,figsize = (10,10))
pyplot.show()

Find out the count of null values for each attributes.

In [ ]:
trainFile.describe(include=[np.object])

In [ ]:
trainFile.isna().sum()

Describe our train dataset. For Object Types (Strings and Timestamps), describe outputs count, unique, top and freq. 

Since the "Survived" column is binary - 0,1 , the mean value is the percentage of the people who survived.

In [ ]:
trainFile[['Pclass','Survived']].groupby(['Pclass'],as_index = False).mean().sort_values(by='Survived', ascending=False)

Sex and Survived Analysis

In [ ]:
trainFile[['Sex','Survived']].groupby(['Sex'],as_index = False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
trainFile[['SibSp','Survived']].groupby(['SibSp'],as_index = False).mean().sort_values(by='Survived', ascending=False)

Calculate the median of Age column and fill the missing values of Age with the median. 

In [ ]:
fillValues = trainFile["Age"].median(axis=0)
fillValues
trainFile['Age'].fillna(fillValues, inplace = True)
trainFile.head(10)



Drop the unwanted columns which do not contribute for analysis.

In [ ]:
#trainFile.drop('Cabin', inplace = True)
trainFile.drop(['PassengerId','Cabin','Ticket'],axis = 1, inplace = True)


In [ ]:
trainFile.drop(['Name'],axis = 1, inplace = True)
trainFile.head(10)

Convert categorical variables into Numeric variables. We have two categorical variables ie. Sex and Embarked. 

In [ ]:
# categoricalVar = ['Sex','Embarked']
# trainFile[categoricalVar] = trainFile[categoricalVar].apply(lambda x: x.astype('category'))
# trainFile[categoricalVar] = trainFile[categoricalVar].apply(lambda x: x.cat.codes)
# trainFile.head(10)

#Using Label Encoder to encode the two categorical variables - Sex and Embarked
encoder1 = LabelEncoder()
encoder2 = LabelEncoder()
trainFile_cat_sex = trainFile['Sex']
trainFile_cat_embarked = trainFile['Embarked']
trainFile_cat_sex_encoded = encoder1.fit_transform(trainFile_cat_sex)
trainFile_cat_embarked_encoded = encoder2.fit_transform(trainFile_cat_embarked)
# print(trainFile_cat_encoded)
# print(encoder.classes_)

In [ ]:
# Now, use One Hot Encoding to assign zeros and ones to each category for sex and embarked attributes
encoder = OneHotEncoder(categories='auto') # categories = 'auto' done to prevent the warning message
trainFile_cat_sex_1hot = encoder.fit_transform(trainFile_cat_sex_encoded.reshape(-1,1))
# print(trainFile_cat_sex_1hot)
trainFile_cat_embarked_1hot = encoder.fit_transform(trainFile_cat_embarked_encoded.reshape(-1,1))
# print(trainFile_cat_embarked_1hot)
# print(type(trainFile_cat_embarked_1hot)) # type is scipy.sparse matrix
trainFile_cat_embarked_1hot.toarray() # converting it to dense 2D a

Split the data into training and test datasets. 


In [ ]:
#Using Linear Regression
X = trainFile.drop('Survived',axis = 1)
y = trainFile['Survived']
print(type(y))
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)
#print(y_test)
#print(y_train)



Apply ML algorithms

Logistic Regression

In [ ]:
clfLogisticRegression = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial').fit(X_train, y_train)
y_pred = clfLogisticRegression.predict(X_test)

In [ ]:
print("Accuracy using Linear Regression Model is: "+ str(accuracy_score(y_test, y_pred)))
print(confusion_matrix(y_test,y_pred))
positives,negatives = confusion_matrix(y_test,y_pred)
TP,FP = positives
FN,TN = negatives
print("Accuracy using confusion matrix is: "+ str((TP+TN)/(TP+FP+FN+TN)))

Decision Tree

In [ ]:
clfDecisionTree = tree.DecisionTreeClassifier()
clfDecisionTree = clfDecisionTree.fit(X_train, y_train)
y_pred = clfDecisionTree.predict(X_test)


In [ ]:
print("Accuracy using Decision Tree Model is: "+ str(accuracy_score(y_test, y_pred)))
print(confusion_matrix(y_test,y_pred))
positives,negatives = confusion_matrix(y_test,y_pred)
TP,FP = positives
FN,TN = negatives
print("Accuracy using confusion matrix is: "+ str((TP+TN)/(TP+FP+FN+TN)))
#acc_decision_tree = round(clfDecisionTree.score(X_test, y_test) * 100, 2)
#print(acc_decision_tree)